In [ ]:
import sympy as sp
import numpy as np
from IPython.display import display, Math, Latex
from sympy.utilities.lambdify import lambdify

# Loading variables

In [ ]:
m_L = sp.symbols(r"m_L")
m_M = sp.symbols(r"m_M")
w_L = sp.symbols(r"w_L")
I_L = sp.symbols(r"I_L")
h_L = sp.symbols(r"h_L")
r = sp.symbols(r"r")
I_M = sp.symbols(r"I_M")

In [ ]:
# Friction coefficients
mu = sp.symbols(r"\mu")
mu_S = sp.symbols(r"\mu_{S}")

# System gains
b_J = sp.symbols(r"b_J")
k_J = sp.symbols(r"k_J")

In [ ]:
# Positions
p_CN = sp.symbols(r"p_{CN}")
p_CT = sp.symbols(r"p_{CT}")
p_MN = sp.symbols(r"p_{MN}")
p_LN = sp.symbols(r"p_{LN}")
p_LT = sp.symbols(r"p_{LT}")
theta_L = sp.symbols(r"\theta_L")
d_T = sp.symbols(r"d_T")
d_N = sp.symbols(r"d_N")

# p_C - p_L
p_LCN = sp.symbols(r"p_{LCN}")
p_LCT = sp.symbols(r"p_{LCT}")

# p_C - p_M
p_MCN = sp.symbols(r"p_{MCN}")
p_MCT = sp.symbols(r"p_{MCT}")

# Velocities
v_MN = sp.symbols(r"v_{MN}")
v_MT = sp.symbols(r"v_{MT}")
v_LN = sp.symbols(r"v_{LN}")
v_LT = sp.symbols(r"v_{LT}")
d_theta_L = sp.symbols(r"\dot\theta_L")
d_theta_M = sp.symbols(r"\dot\theta_M")
d_d_T = sp.symbols(r"\dot{d}_T")
d_d_N = sp.symbols(r"\dot{d}_N")

In [ ]:
# Input forces
F_GT = sp.symbols(r"F_{GT}")
F_GN = sp.symbols(r"F_{GN}")
F_OT, F_ON, tau_O = sp.symbols(r"F_{OT}, F_{ON} \tau_O")

# Control inputs
dd_theta_Ld = sp.symbols(r"\ddot\theta_{Ld}")
dd_d_Nd = sp.symbols(r"\ddot{d}_{Nd}")
dd_d_Td = sp.symbols(r"\ddot{d}_{Td}")
dd_theta_Md = sp.symbols(r"\ddot\theta_{Md}")

outputs = [
    a_LT, a_MT, a_MN, F_NM, F_FL, F_FM, F_NL, F_CT, F_CN, tau_M, dd_theta_L, a_LN, dd_theta_M, dd_d_N, dd_d_T
] = sp.symbols(
    r"a_{LT}, a_{MT}, a_{MN}, F_{NM}, F_{FL}, F_{FM}, F_{NL}, F_{CT}, F_{CN}, \tau_M, \ddot\theta_L, a_{LN}, \ddot\theta_M, \ddot{d}_N, \ddot{d}_T"
)

outputs = list(outputs)

## Differentiation of $\vec d$

In [ ]:
t = sp.symbols("t")
theta_L_func = sp.Function(r'\theta_L')(t)
N_hat = sp.Function(r'\hat N')(theta_L_func)
T_hat = sp.Function(r'\hat T')(theta_L_func)

d_T_func = sp.Function(r"d_T")(t)
d_N_func = sp.Function(r"d_N")(t)
d_g = d_T_func*T_hat + d_N_func*N_hat

d_vel_g = sp.diff(d_g, t)

d_vel_g = d_vel_g.subs(sp.diff(N_hat, t), -
                       sp.diff(theta_L_func, t)*T_hat)
d_vel_g = d_vel_g.subs(
    sp.diff(T_hat, t), sp.diff(theta_L_func, t)*N_hat)

d_acc_g = sp.diff(d_vel_g, t)
d_acc_g = d_acc_g.subs(sp.diff(N_hat, t), -
                       sp.diff(theta_L_func, t)*T_hat)
d_acc_g = d_acc_g.subs(
    sp.diff(T_hat, t), sp.diff(theta_L_func, t)*N_hat)

d_acc_cos_g = d_acc_g
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(theta_L_func, t, t), dd_theta_L)
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(d_T_func, t, t), dd_d_T)
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(d_N_func, t, t), dd_d_N)
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(theta_L_func, t), d_theta_L)
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(d_T_func, t), d_d_T)
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(d_N_func, t), d_d_N)
d_acc_cos_g = d_acc_cos_g.subs(d_T_func, d_T)
d_acc_cos_g = d_acc_cos_g.subs(d_N_func, d_N)

In [ ]:
dd_d_g_T = d_acc_cos_g.subs(N_hat, 0).subs(T_hat, 1)
dd_d_g_T

In [ ]:
dd_d_g_N = d_acc_cos_g.subs(T_hat, 0).subs(N_hat, 1)
dd_d_g_N

In [ ]:
p_M_func = sp.Function(r"p_M")(t)
p_L_func = sp.Function(r"p_L")(t)
v_M = sp.symbols(r"v_M")
v_L = sp.symbols(r"v_L")
d_s = (p_M_func + r*N_hat) - (p_L_func + (w_L/2)*T_hat - (h_L/2)*N_hat)

d_vel_s = sp.diff(d_s, t)
d_vel_s = d_vel_s.subs(sp.diff(N_hat, t), -
                       sp.diff(theta_L_func, t)*T_hat)
d_vel_s = d_vel_s.subs(
    sp.diff(T_hat, t), sp.diff(theta_L_func, t)*N_hat)

d_acc_s = sp.diff(d_vel_s, t)
d_acc_s = d_acc_s.subs(sp.diff(N_hat, t), -
                       sp.diff(theta_L_func, t)*T_hat)
d_acc_s = d_acc_s.subs(
    sp.diff(T_hat, t), sp.diff(theta_L_func, t)*N_hat)

d_acc_cos_s = d_acc_s
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(theta_L_func, t, t), dd_theta_L)
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(d_T_func, t, t), dd_d_T)
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(d_N_func, t, t), dd_d_N)
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(theta_L_func, t), d_theta_L)
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(d_T_func, t), d_d_T)
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(d_N_func, t), d_d_N)
d_acc_cos_s = d_acc_cos_s.subs(d_T_func, d_T)
d_acc_cos_s = d_acc_cos_s.subs(d_N_func, d_N)

In [ ]:
dd_d_s_T = d_acc_cos_s.subs(N_hat, 0).subs(T_hat, 1)
dd_d_s_T = dd_d_s_T.subs(sp.diff(p_M_func, t, t), a_MT)
dd_d_s_T = dd_d_s_T.subs(sp.diff(p_L_func, t, t), a_LT)
dd_d_s_T

In [ ]:
dd_d_s_T.subs(sp.diff(p_M_func, t, t), a_MT)

In [ ]:
dd_d_s_N = d_acc_cos_s.subs(T_hat, 0).subs(N_hat, 1)
dd_d_s_N = dd_d_s_N.subs(sp.diff(p_M_func, t, t), a_MN)
dd_d_s_N = dd_d_s_N.subs(sp.diff(p_L_func, t, t), a_LN)
dd_d_s_N

# Setting up equations and matrices

In [ ]:
nat_eqs = [
    # Link tangential force balance
    [m_L*a_LT, F_FL+F_GT+F_OT],
    # Link normal force balance
    [m_L*a_LN, F_NL + F_GN + F_ON, ],
    # Manipulator tangential force balance
    [m_M*a_MT, F_FM + F_CT, ],
    # Manipulator normal force balance
    [m_M*a_MN, F_NM+F_CN, ],
    # Link moment balance
    [I_L*dd_theta_L, (-w_L/2)*F_ON - (p_LCN) * \
     F_FL + (p_LCT)*F_NL + tau_O, ],
    # Manipulator moment balance
    [I_M*dd_theta_M, tau_M-F_FM*(p_MCN), ],
    # 3rd law normal forces
    [F_NL, -F_NM],
    # Friction relationship L
    [F_FL, mu*mu_S*F_NL],
    # Friction relationship M
    [F_FM, -F_FL],
    # d_T derivative is derivative
    [dd_d_s_T, dd_d_g_T],
    # d_N derivative is derivative
    [dd_d_s_N, dd_d_g_N],
    # dd_d_N = 0 in real life assuming we're in contact
    [dd_d_N, 0],
]

art_eqs = [
#     [dd_d_N, dd_d_Nd],
#     [dd_d_T, dd_d_Td],
#     [dd_theta_L, dd_theta_Ld],
#     [dd_theta_M, dd_theta_Md],
]

env_eqs = nat_eqs + art_eqs

In [ ]:
dd_d_s_N

In [ ]:
out_str = r"\begin{aligned}"
for eq in env_eqs:
    out_str += sp.latex(eq[0]) + r" &= " + sp.latex(eq[1]) + r" \\" + "\n"
out_str += "\end{aligned}"
display(Math(out_str))

In [ ]:
env_eqs[10][0]-a_MN

In [ ]:
env_eqs[7]

In [ ]:
eq_idxs = {3, 10, 1, 6, 7, 4}
out_str = r"\begin{aligned}"
for i, eq in enumerate(env_eqs):
    if i in eq_idxs:
        out_str += sp.latex(eq[0]) + r" &= " + sp.latex(eq[1]) + r" \\" + "\n"
out_str += "\end{aligned}"
display(Math(out_str))

In [ ]:
A = []
b = []
for lhs, rhs in env_eqs:
    A_row = []
    b_term = rhs - lhs
    for output_term in outputs:
        try:
            coeff_L = lhs.coeff(output_term)
        except AttributeError:
            coeff_L = 0
        try:
            coeff_R = rhs.coeff(output_term)
        except AttributeError:
            coeff_R = 0
        coeff = coeff_L - coeff_R
        A_row.append(coeff)
        b_term += coeff * output_term
    A.append(A_row)
    b.append(b_term)
A = sp.SparseMatrix(A)
A.simplify()
b = sp.Matrix([b]).T
b.simplify()
x = sp.Matrix([outputs]).T
x.simplify()

In [ ]:
A_aug = A.row_join(b)
results = A_aug.rref()[0]
A_prime = results[:, :-1]
b_prime = results[:, -1]

In [ ]:
A_prime

In [ ]:
A_prime@x

# Dynamics
For each equation, we want $\alpha_\theta(t,\cdot)\ddot\theta_L+\alpha_x(t,\cdot)\ddot x + \beta(t,\cdot)u = \gamma(t,\cdot)$, where $x$ is the thing we're trying to control and $u$ is the control input.
## Simplify coefficients
### $\alpha/\beta$ terms

In [ ]:
p_MLN = sp.symbols("p_{MLN}")
p_MLN

In [ ]:
(A_prime@x)

In [ ]:
lhss = (A_prime@x)[-5:-2,:]
lhss

In [ ]:
rhss = (b_prime)[-5:-2,:]
rhss

## $F_{CN}$

In [ ]:
lhss

In [ ]:
term = lhss[1].coeff(a_LN)
term = term.expand()
term = term.collect(mu)
term